In [61]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
# Set the absolute path to the `src` directory
src_path = '/Users/milicapajkic/Library/CloudStorage/OneDrive-HochschuleLuzern/HSLU/MasterThesis/Unlimiformer_train/src'
sys.path.append(src_path)

In [9]:
from unlimiformer import Unlimiformer
from random_training_unlimiformer import RandomTrainingUnlimiformer
from usage import UnlimiformerArguments, training_addin

from transformers import BartForConditionalGeneration, AutoTokenizer
from datasets import load_dataset
import torch

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [48]:
from huggingface_hub import login
login()

In [54]:

# If the dataset is gated/private, make sure you have run huggingface-cli login
#dataset = load_dataset("mimiklee/masterthesis_train")
dataset = load_dataset("mimiklee/masterthesis_data")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [55]:
dataset

DatasetDict({
    train: Dataset({
        features: ['paperId', 'title', 'year', 'publicationTypes', 'Abstract', 'All_Text_with_Titles', 'len_text', 'len_abstract'],
        num_rows: 68243
    })
    validation: Dataset({
        features: ['paperId', 'title', 'year', 'publicationTypes', 'Abstract', 'All_Text_with_Titles', 'len_text', 'len_abstract'],
        num_rows: 14624
    })
    test: Dataset({
        features: ['paperId', 'title', 'year', 'publicationTypes', 'Abstract', 'All_Text_with_Titles', 'len_text', 'len_abstract'],
        num_rows: 14624
    })
})

In [13]:
# example using govreport
modelname = "abertsch/unlimiformer-bart-govreport-alternating"

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained(modelname)

/Users/milicapajkic/Library/CloudStorage/OneDrive-HochschuleLuzern/HSLU/MasterThesis/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['PDF_Name', 'Abstract', 'All_Text_with_Titles', 'language'],
        num_rows: 43271
    })
})

In [136]:
example_input = dataset['train'][10000]['All_Text_with_Titles']
example_output = dataset['train'][10000]['Abstract']

In [137]:
example = tokenizer(example_input, truncation=False, return_tensors="pt")
truncated_example = tokenizer(example_input, truncation=True, max_length=1024, return_tensors="pt")

example.to(device)
truncated_example.to(device)

print(f"INPUT LENGTH (tokens): {example['input_ids'].shape[-1]}")

In [39]:
defaults = UnlimiformerArguments()
unlimiformer_kwargs = {
            'layer_begin': defaults.layer_begin, 
            'layer_end': defaults.layer_end,
            'unlimiformer_head_num': defaults.unlimiformer_head_num, 
            'exclude_attention': defaults.unlimiformer_exclude, 
            'chunk_overlap': defaults.unlimiformer_chunk_overlap,
            'model_encoder_max_len': defaults.unlimiformer_chunk_size,
            'verbose': defaults.unlimiformer_verbose, 'tokenizer': tokenizer,
            'unlimiformer_training': defaults.unlimiformer_training,
            'use_datastore': defaults.use_datastore,
            'flat_index': defaults.flat_index,
            'test_datastore': defaults.test_datastore,
            'reconstruct_embeddings': defaults.reconstruct_embeddings,
            'gpu_datastore': defaults.gpu_datastore,
            'gpu_index': defaults.gpu_index
}

In [40]:
model.to(device)
# the output of the model /without/ using unlimiformer 
truncated_out = tokenizer.batch_decode(model.generate(**truncated_example, max_length=512))

In [41]:
truncated_out = tokenizer.batch_decode(model.generate(**example, max_length=512))

In [42]:
model = Unlimiformer.convert_model(model, **unlimiformer_kwargs)
model.eval()
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): Laye

In [43]:
# the output of the model /with/ unlimiformer 
unlimiformer_out = tokenizer.batch_decode(model.generate(**example, max_length=512), ignore_special_tokens=True)[0]
print(unlimiformer_out)

</s><s>If American journalism were a religion, as it has been called from time to time, its supreme deity would be 'objectivity'. As journalism has turned more professionalized over the last decades, the concept of objectivity has become significantly enshrined in media policies, newsroom routines, and journalism education. This paper is one of a number of CRS products focusing on objectivity and political pluralism in American journalism.
This report is intended to provide an overview of the objectivity concept and its relationship to social democracy in the post-Cold War era. It is designed to demonstrate that objectivity, as defined by a journalist's " god-terms," is a set of ethical guidelines that journalists use in dealing with a wide range of issues, including how to arrive at an unbiased, socially diverse, and pluralistic public discourse.
If American journalists were to strive to be a religious denomination, their highest Deity would be' objectivity. The aim of this paper is t

In [47]:
print(example_output)

This paper starts from the observation that the concept of objectivity, along with its twin sentries 'balance' and 'impartiality', is generally regarded as a cornerstone of journalism and, consequently, of journalism research. The aim of this paper is to show that the analytical ideal of objectivity, instead of enabling, in fact inhibits media pluralism research. The first section focuses on unveiling the ideological nature of this ideal by relating it both socio-historically and analytically to a post-ideological and consensual understanding of society. Since we find this ideal only allowing for the evaluation of journalism within the limits of social consensus (pluralism 'within the box'), the second section seeks for alternative analytical concepts to evaluate journalism about and beyond the limits of social consensus (pluralism 'outside the box'). To illustrate the difference between both approaches, the popular concept of partisan media bias is juxtaposed to the alternative framew